In [19]:
import pandas as pd

df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
df = df.drop(['CustomerId', 'Surname', 'RowNumber'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

X = df.drop(['Exited'], axis=1)
y = df['Exited']

numerical_columns = X.select_dtypes(exclude='object').columns
categorical_columns = X.select_dtypes('object').columns

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical_columns),
        ('scaler', StandardScaler(), numerical_columns)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

In [22]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
# Setting up TensorBoard and Early Stopping callbacks
log_dir = 'logs/fit'
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
model.save('model.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                896       
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5121 (20.00 KB)
Trainable params: 5121 (20.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


/Users/bijonlahiri/Desktop/ann_classification_project_churn_modelling/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
# Train the model
history = model.fit(
    X_train_preprocessed,
    y_train,
    validation_data=(X_test_preprocessed, y_test),
    epochs=100,
    batch_size=10,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
800/800 [==============================] - 5s 4ms/step - loss: 0.4048 - accuracy: 0.8339 - val_loss: 0.3778 - val_accuracy: 0.8420
Epoch 2/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3490 - accuracy: 0.8584 - val_loss: 0.3711 - val_accuracy: 0.8495
Epoch 3/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3384 - accuracy: 0.8624 - val_loss: 0.3650 - val_accuracy: 0.8455
Epoch 4/100
800/800 [==============================] - 3s 4ms/step - loss: 0.3330 - accuracy: 0.8649 - val_loss: 0.3721 - val_accuracy: 0.8515
Epoch 5/100
800/800 [==============================] - 2s 3ms/step - loss: 0.3302 - accuracy: 0.8634 - val_loss: 0.3572 - val_accuracy: 0.8545
Epoch 6/100
800/800 [==============================] - 3s 3ms/step - loss: 0.3269 - accuracy: 0.8643 - val_loss: 0.3561 - val_accuracy: 0.8515
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 0.3228 - accuracy: 0.8699 - val_loss: 0.3600 - val_accuracy: 0.8560

In [25]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [26]:
%load_ext tensorboard

In [28]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 9220), started 0:25:02 ago. (Use '!kill 9220' to kill it.)